# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [2]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [7]:
import pandas as pd

# Define the tables and their descriptions
data = {
    'table': ['employees', 'salary', 'studies'],
    'definition': [
        'Employee information, name, position, department, and start date.',
        'Salary details for each year, including base, bonus, and adjustments.',
        'Educational studies, name of the institution, degree, and graduation year.'
    ]
}

df = pd.DataFrame(data)
print(df)


       table                                         definition
0  employees  Employee information, name, position, departme...
1     salary  Salary details for each year, including base, ...
2    studies  Educational studies, name of the institution, ...


In [8]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [9]:
print(text_tables)

employees: Employee information, name, position, department, and start date.
salary: Salary details for each year, including base, bonus, and adjustments.
studies: Educational studies, name of the institution, degree, and graduation year.


In [10]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [13]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question= "Which employees have a master's degree and started after 2015?")#ENTER YOUR QUERY HERE)

In [14]:
print(return_OAI(pqt1))

{
    "tables": ["employees", "studies"]
}


In [ ]:
prompt_question_tables = """
Given the following tables and their full definitions:

### Tables
{tables}

From the user's question below, identify which tables are needed to write the SQL query.

Return your answer as a JSON object like this:
{{
    "tables": {{
        "table_name": "short description or label from the full definition"
    }}
}}

Only include relevant tables and use a short title based on each table's description.

### User Question:
{question}
"""


In [27]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question="Find employees who earned a bonus in 2022 and also have a postgraduate degree.")#ENTER YOUR QUERY HERE)

In [28]:
print(return_OAI(pqt3))

{
    "tables": {
        "employees": "Employee information, name, position, department, and start date.",
        "salary": "Salary details for each year, including base, bonus, and adjustments.",
        "studies": "Educational studies, name of the institution, degree, and graduation year."
    }
}


In [29]:
prompt_question_tables = """
Given the following tables and their full definitions:

### Tables
{tables}

From the user's question below, identify which tables are needed to write the SQL query.

Return your answer as a JSON object like this:
{{
    "tables": {{
        "table_name": "short label or title for that table (not the full definition)"
    }}
}}

Use only the short label or topic of each table, not the full description. Be concise.

### User Question:
{question}
"""


In [30]:
pqt3 = prompt_question_tables.format(
    tables=text_tables,
    question= "Find employees who earned a bonus in 2022 and also have a postgraduate degree."
)

print(return_OAI(pqt3))


{
    "tables": {
        "employees": "Employee information",
        "salary": "Salary details",
        "studies": "Educational studies"
    }
}


In [32]:
prompt_find_tables_explained = """
Given the following table descriptions:

{tables}

Based on the user's question below, list only the tables that are needed to answer it using SQL.
For each table, provide a short one-line reason why it is needed.

### User Question:
{question}

Return the result as a JSON object like this:
{{
    "tables": {{
        "table_name": "short reason why this table is required"
    }}
}}
"""

pqt4 = prompt_find_tables_explained.format(
    tables=text_tables,
    question="List employees hired after 2015 with a master's degree."
)

print(return_OAI(pqt4))

{
    "tables": {
        "employees": "To filter employees hired after 2015",
        "studies": "To check for employees with a master's degree"
    }
}


In [33]:
prompt_explain_query = """
You are a helpful assistant that understands database structures.

Here are the available tables:

{tables}

Based on the following user request, explain in plain English what kind of data should be retrieved and how the tables are related.

### User Question:
{question}

Provide your explanation in 2-3 sentences.
"""

pqt5 = prompt_explain_query.format(
    tables=text_tables,
    question="Get employees with a bonus greater than 1000 and a PhD."
)

print(return_OAI(pqt5))


To retrieve employees with a bonus greater than 1000 and a PhD, we need to query the `employees` table to get the employee information and then join it with the `salary` table to check for a bonus greater than 1000. Additionally, we need to join the `studies` table to filter employees with a PhD degree. The relationship between the tables is established through a common identifier, such as employee ID.


In [34]:
prompt_improve_question = """
Given the following table descriptions:

{tables}

Here is a user's question about the database:

### User Question:
{question}

Suggest one or two clarifying follow-up questions the user could answer to improve the precision of their SQL query.
Respond as a list of questions.
"""

pqt6 = prompt_improve_question.format(
    tables=text_tables,
    question="Show employees who earned a high bonus."
)

print(return_OAI(pqt6))


1. What criteria define a "high bonus" in this context? Is it a specific amount or a percentage of the base salary?
2. Are there any specific years or time periods for which the bonus should be considered, or should we look at all available data in the salary table?


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

## GPT SQL Reasoning Evaluation Report
I tested three different prompt styles to explore how well GPT identifies relevant tables, explains queries in plain English, and suggests follow-up clarifying questions.

1. Prompt: "Which tables are needed?"

Result:
{
  "tables": {
    "employees": "To filter employees hired after 2015",
    "studies": "To check for employees with a master's degree"
  }
}

Evaluation: GPT correctly identified only the relevant tables and gave short, logical reasons for each. The format matched expectations and there were no hallucinations.


2. Prompt: "Explain the query in plain English"

Result: GPT clearly described the data retrieval process using joins across employees, salary, and studies, explaining how to filter for a bonus and a PhD.

Evaluation: The explanation was clear and accurate. It correctly inferred relationships and logical query steps. No issues found.


3. Prompt: "Suggest clarifying follow-up questions"

Result: GPT returned:

What defines a "high bonus"?

Are there specific years or time periods?

Evaluation: Both follow-up questions were insightful and relevant, helping refine the query. This prompt worked well for improving precision.

## What I Learned
- GPT performs well with structured prompts and gives consistent, accurate results when formatting is clearly defined.

- It’s useful for breaking down SQL logic and encouraging query clarity.

- Prompt wording matters — being specific about format (short label vs. full description) makes a big difference.